# # AI Workshop - Lab 3-1b: Scheduling
## Employee Scheduling Optimization Using Google OR-Tools

This notebook demonstrates how to solve a common scheduling problem using **Google OR-Tools**, a powerful library for optimization. In this example, we will create a schedule for employees that satisfies their individual working hour constraints while ensuring that shifts are adequately staffed.

---

## **Problem Overview**

A company needs to create a weekly schedule for its employees. The scheduling must respect two sets of constraints:

1. **Employee Constraints**:
   - **Minimum shifts**: Each employee must work at least a specified number of shifts during the week.
   - **Maximum shifts**: Each employee cannot work more than a specified number of shifts during the week.

2. **Shift Requirements**:
   - Each shift must have a minimum number of employees working to meet operational requirements.

---

### **Goal**

The goal is to create a schedule that ensures:
- All employees are scheduled within their constraints.
- All shifts are adequately staffed with the minimum required number of employees.

We will use Google OR-Tools to model and solve this problem efficiently.

In [ ]:
!pip install -q ortools

In [ ]:
from ortools.linear_solver import pywraplp

## Problem Data

In this section, we define the **problem data** that serves as the foundation for our scheduling model. This includes the employees, their scheduling constraints, the available shifts, and the minimum staffing requirements for each shift.

---

### **Components of the Data**

1. **Employees**:
   A list of employees, each with:
   - **Minimum shifts**: The least number of shifts they must work in a week.
   - **Maximum shifts**: The most number of shifts they can work in a week.

2. **Shifts**:
   A list of all shifts that need to be scheduled. Each shift is associated with a specific day and time (e.g., "Monday Morning", "Tuesday Evening").

3. **Minimum Staffing Requirements**:
   A dictionary specifying the minimum number of employees required for each shift. For simplicity, we assume all shifts require at least 2 employees, but you can customize these values.

In [ ]:
# Data

# Employees and their constraints
employees = {
    'Alice':   {'min_shifts': 2, 'max_shifts': 14},
    'Bob':     {'min_shifts': 3, 'max_shifts': 15},
    'Charlie': {'min_shifts': 2, 'max_shifts': 13},
    'David':   {'min_shifts': 4, 'max_shifts': 5},
    'Eve':     {'min_shifts': 3, 'max_shifts': 14},
    'Frank':   {'min_shifts': 2, 'max_shifts': 3},
}

# Shifts for one week (Monday to Friday, 3 shifts per day)
shifts = [
    'Monday Morning',    'Monday Afternoon',    'Monday Evening',
    'Tuesday Morning',   'Tuesday Afternoon',   'Tuesday Evening',
    'Wednesday Morning', 'Wednesday Afternoon', 'Wednesday Evening',
    'Thursday Morning',  'Thursday Afternoon',  'Thursday Evening',
    'Friday Morning',    'Friday Afternoon',    'Friday Evening',
]

# Minimum number of employees required per shift
min_staffing = {}
for shift in shifts:
    min_staffing[shift] = 2

## Defining the Model

In this section, we create an **integer programming model** to solve the employee scheduling problem. The model uses binary variables to represent whether an employee is assigned to a particular shift. The constraints ensure that:
1. Each employee's total assigned shifts stay within their **minimum and maximum limits**.
2. Each shift has at least the **required minimum number of employees**.

We will use **Google OR-Tools** with the `SCIP` solver, which is ideal for integer programming problems.

---

### **Key Concepts**

1. **Decision Variables:**
   For each employee and each shift, we create a binary variable:
   - `1`: The employee is assigned to the shift.
   - `0`: The employee is not assigned to the shift.

2. **Constraints:**
   - **Employee Constraints:** Ensure the total shifts assigned to each employee are within their specified limits.
   - **Shift Constraints:** Ensure each shift has enough employees to meet the minimum staffing requirement.

3. **Solver:**
   We use the `SCIP` solver from OR-Tools, which is optimized for integer programming.

In [ ]:
# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')  # SCIP is suitable for integer programming

## Defining Decision Variables

The **decision variables** in this model represent whether an employee is assigned to a specific shift. These variables are binary, meaning they can only take two values:
- `1`: The employee is assigned to the shift.
- `0`: The employee is not assigned to the shift.

By defining these variables for every employee and shift combination, we create the foundation of the scheduling model. These variables will later be used to enforce constraints and calculate the optimal schedule.

---

### **Key Points**

1. **Binary Representation:**
   The decision variables are binary because an employee is either working a shift or not. There are no partial assignments.

2. **Why Use Binary Variables?**
   Binary variables make it easier to model yes/no decisions, such as whether an employee works a shift.

3. **Structure:**
   We define a binary variable for every possible combination of employees and shifts. For example:
   - `variables['Alice', 'Monday Morning']`: Represents whether Alice is assigned to the Monday morning shift.

Essentially, we are creating the following table for the model to use:

| Employee | Monday Morning | Monday Afternoon | ... | Friday Evening |
|----------|----------------|------------------|-----|----------------|
| Alice    | 0              | 1                | ... | 0              |
| Bob      | 1              | 0                | ... | 1              |
| ...      | ...            | ...              | ... | ...            |
| Frank    | 0              | 0                | ... | 1              |

In [ ]:
# Decision variables
variables = {}
for employee in employees:
    for shift in shifts:
        variables[employee, shift] = solver.BoolVar(f'{employee}_{shift}')

## Defining Constraints

Constraints are critical in the scheduling model to ensure that the solution respects employee limits and staffing requirements. In this section, we define two types of constraints:

---

### **1. Employee Hour Constraints**

These constraints ensure that each employee works within their specified range of shifts:
- **Minimum Shifts:** Each employee must work at least their defined minimum number of shifts.
- **Maximum Shifts:** Each employee cannot work more than their defined maximum number of shifts.

Mathematically, we represent these constraints using the total shifts assigned to each employee:

$$
\text{Minimum Shifts} \leq \text{Total Shifts for Employee} \leq \text{Maximum Shifts}
$$

---

### **2. Minimum Staffing Constraints**

These constraints ensure that each shift is adequately staffed:
- **Minimum Employees per Shift:** Each shift must have at least the required number of employees assigned to it, as specified in the `min_staffing` dictionary.

Mathematically, this constraint can be represented as:

$$
\text{Total Employees Assigned to Shift} \geq \text{Minimum Staffing for Shift}
$$


In [ ]:
# Constraints

# Employee Hour Constraints: ensure employees work between their minimum and maximum shifts
for employee, constraints in employees.items():
    # Total shifts assigned to the employee
    total_shifts = sum(variables[employee, shift] for shift in shifts)
    # Minimum shifts constraint
    solver.Add(total_shifts >= constraints['min_shifts'])
    # Maximum shifts constraint
    solver.Add(total_shifts <= constraints['max_shifts'])

# Minimum Staffing Constraints: ensure each shift has enough employees
for shift in shifts:
    # Total employees assigned to the shift
    total_staffing = sum(variables[employee, shift] for employee in employees)
    # Minimum staffing constraint
    solver.Add(total_staffing >= min_staffing[shift])

## Defining the Objective Function

The **objective function** specifies the goal of the optimization. In this scheduling problem, the goal is to **balance the workload among employees** by minimizing the maximum number of shifts assigned to any single employee. This ensures fairness in the schedule and avoids overburdening certain employees.

---

### **Key Idea**

1. Define a variable, `max_workload`, to represent the maximum number of shifts assigned to any single employee.
2. Add constraints to ensure that the total shifts assigned to each employee do not exceed `max_workload`.
3. Minimize `max_workload`, thereby balancing the workload across all employees.

In [ ]:
# Objective: Minimize the maximum workload
max_workload = solver.NumVar(0, solver.infinity(), 'max_workload')  # Variable for maximum workload

# Add constraints to ensure total shifts per employee do not exceed max_workload
for employee in employees:
    total_shifts = sum(variables[employee, shift] for shift in shifts)
    solver.Add(total_shifts <= max_workload)

# Set the objective to minimize max_workload
solver.Minimize(max_workload)

## Solve the Model

Let's run the solver and check if it finds an optimal solution.

In [ ]:
# Solve the model
status = solver.Solve()

# Check if the problem has an optimal solution
if status == pywraplp.Solver.OPTIMAL:
    print('Optimal Solution Found')
else:
    print('The problem does not have an optimal solution')


## Results

If an optimal solution is found, we can extract the schedule for each employee and check the staffing levels for each shift.

In [ ]:
# Build a table of the schedule
schedule = {employee: {} for employee in employees}

# Extract the schedule from the variables
for employee, shift in variables:
    if variables[employee, shift].solution_value() > 0.5:
        schedule[employee][shift] = "Assigned"

# Print the schedule
import pandas as pd

df_schedule = pd.DataFrame(schedule).fillna("")
df_schedule.style.set_caption('Employee Schedule')

df_schedule

## **Conclusion**

In this notebook, we demonstrated how to use **Google OR-Tools** to solve a common scheduling problem efficiently. By defining appropriate constraints and an objective function, we created a fair and optimal schedule for employees while meeting the minimum staffing requirements for each shift.

This approach can be extended to more complex scheduling scenarios and additional constraints, making it a versatile tool for workforce management and optimization.